### Import Dependencies

In [1]:
from __future__ import print_function
import traceback
import time
import paramiko
from paramiko import SSHClient
from paramiko_expect import SSHClientInteraction
import sys
import os
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")
    
from Driver_TECSource_5240 import *
from Driver_OSA_MS9740B import *

In [2]:
rm = pyvisa.ResourceManager()

tec = TECSource_5240()
tec.open(rm,'ASRL7::INSTR')

osa = OSA_MS9740B()
osa.open(rm)

smu = rm.open_resource('ASRL5::INSTR')
smu.baud_rate = 38400

Failed to connect to TEC. Verify Address


NameError: name 'exit' is not defined

### Class to SSH into Zeus

In [31]:
class SSH_to_ZEUS_class:
    client: SSHClient
    def open_session(self, HOSTNAME_PYNQ='pynq not set'):   # example 'pynq4'
        HOSTNAME = HOSTNAME_PYNQ   # example 'pynq4'
        USERNAME = 'xilinx'
        PASSWORD = 'xilinx'
        sudo_prompt = ".*\#\s+"
        PROMPT = '.*\$\s+'
        
        # Use SSH client to login
        try:
            # Create a new SSH client object
            client = paramiko.SSHClient()
            self.client = client

            # Set SSH key parameters to auto accept unknown hosts
            client.load_system_host_keys()
            client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

            if True:
                client.connect(hostname=HOSTNAME, username=USERNAME, password=PASSWORD)
            if False:
                mySSHK = '/path/to/sshkey.pub'
                mySSHK = '/id_rsa_pynq9.pub'
                mySSHK = 'id_rsa_pynq9.pub'
                mySSHK = r'C:\Users\cdavies\.ssh\id_rsa.pub'  # this worked consistently Jan 29
                client.connect(hostname=HOSTNAME, username=USERNAME, key_filename=mySSHK)

            # Create a client interaction class which will interact with the host
            interact = SSHClientInteraction(client, timeout=6, display=False)
            self.interact = interact

            time.sleep(0.1)  # do not remove
            print("\n\n\nsending login for sudo-------- \n")
            interact.send(f"sudo -i")
            time.sleep(0.1)  # do not remove

            interact.send('xilinx')
            interact.expect([PROMPT, sudo_prompt, 'password for xilinx.*\:'])
            interact.send(r'source /usr/local/share/pynq-venv/bin/activate')
            interact.expect([PROMPT, sudo_prompt, 'password for xilinx.*\:'])
            PYTHON_PROMPT = r'(>>> )'
            interact.send('python3')
            time.sleep(0.3)  # wait for python to come up
            interact.expect([PYTHON_PROMPT, '/[>]', PROMPT, sudo_prompt])
            interact.send(r"import time")
            interact.expect(PYTHON_PROMPT)
            
            interact.send('from artemis2 import *')
            interact.expect(PYTHON_PROMPT)
            interact.send('power.bypass_asic_check(True)')
            interact.expect(PYTHON_PROMPT)
            interact.send('power.init(TargetRate.Gbps56_0)')
            interact.expect(PYTHON_PROMPT) 
            
            if False:
                print(interact.current_output_clean)
                
        except Exception:
            traceback.print_exc()
        self.interact = interact

    def write_only(self, command="pwd"):
        PYTHON_PROMPT = r'(>>> )'
        self.interact.send(rf"{command}")
        self.interact.expect(PYTHON_PROMPT)

    def write_read(self, command="pwd"):
        PYTHON_PROMPT = r'(>>> )'
        self.interact.send(rf"{command}")
        answer = self.interact.expect(PYTHON_PROMPT)
        return answer

    def get_light_engine_temperatures(self):
        self.write_read("temperature.print_all()")
        readframe = self.interact.current_output_clean
        Ambient_C = float(readframe.split(' ')[1].split('°C')[0])
        LE_C = float(readframe.split(' ')[4].split('°C')[0])
        return Ambient_C, LE_C

    def get_mpd_readout(self, Channel):
        query_string = "adc.LE_MPD_IMON_" + str(Channel) + ".print()"
        self.write_read(query_string)
        answer = self.interact.current_output_clean
        mpd_mA = float(answer.split(' ')[1].split('°C')[0])
        return mpd_mA
    
    def close(self):
        try:
            self.client.close()   # we dont want to close
        except Exception:
            print("trouble closing ssh client")

### Setup Measurement Conditions

In [41]:
T_Start = 55
T_Stop = 55
T_Step = 10
t_wait = 0
n = int(((T_Stop-T_Start)/T_Step)+1)

wavelength_start = 1565
wavelength_stop = 1585
wavelength_points = 2001
OSA_Resolution = 0.03
OSA_VBW = 1000
timeout = 10000

bias_start = 0
bias_stop = 500
bias_step = 1
bias_current = np.linspace(bias_start,bias_stop,int((bias_stop-bias_start)/bias_step) + 1)

Light_engine = "241323"
Channel = 4

osa.set_timeout(float('+inf'))

### Function to Sweep OSA

In [42]:
def sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_start, wavelength_stop, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_start,wavelength_stop,wavelength_points)
    Wavelength_peak_nm, Power_peak_dBm = osa.get_peak()   
    if Power_peak_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

def read_voltage():
    smu.write(f":SOUR:FUNC CURR")
    smu.write(f":SOUR:CURR:MODE FIXED")
    smu.write(r':SENS:FUNC "VOLT"')
    smu.write(f":SOUR:CURR:RANG MIN")
    smu.write(f":SOUR:CURR:LEV 0")
    smu.write(f":SENS:VOLT:PROT 10")
    smu.write(f":OUTP ON")
    smu.write(f":READ?")
    Voltage_V = float(smu.read())
    smu.write(f":OUTP OFF")
    return Voltage_V

### Measurement Routine

In [43]:
SSH_ZEUS = SSH_to_ZEUS_class()
SSH_ZEUS.open_session('pynq1')
SSH_ZEUS.write_read("fan.set_le_duty_cycle(90)")




sending login for sudo-------- 



0

In [44]:
meas_df = pd.DataFrame()

In [ ]:
for k in range(0,n,1):
    T_Set = T_Start+k*T_Step
    tec.set_temperature(T_Set)
    tec.set_output_on()
    time.sleep(t_wait)
    for j in range(0,int(((bias_stop-bias_start)/bias_step)+1),bias_step): 
        query_string = "light_engine.set_laser_ma(LEChannel.LE" + str(Channel) + "," + str(bias_current[j]) + ")"
        SSH_ZEUS.write_read(query_string)
        T_read = tec.get_temperature()
        Voltage_V = read_voltage()
        Ambient_C, LE_C = SSH_ZEUS.get_light_engine_temperatures()
        Mpd_mA = SSH_ZEUS.get_mpd_readout(Channel)
        Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW)
        meas_df_temp = pd.DataFrame({'Light_engine':Light_engine, 'Channel':Channel, 'Bias_current_mA':bias_current[j], 'Voltage_V':Voltage_V, 'TEC_C':T_read, 'Ambient_C':Ambient_C, 'LE_C':LE_C, 'MPD_mA':Mpd_mA, 'Wavelength_nm':Wavelength_list, 'Power_dBm':Power_list, 
                                    'Power_uW':Power_list_uW, 'Wavelength_peak_nm':Wavelength_peak_nm, 'Power_peak_nm':Power_peak_dBm, 'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                                    '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
        data = [meas_df_temp, meas_df]
        meas_df = pd.concat(data,ignore_index=True,sort=False)
        print("Bias: "+str(j)+" mA")

Bias: 0 mA
Bias: 1 mA
Bias: 2 mA
Bias: 3 mA
Bias: 4 mA
Bias: 5 mA
Bias: 6 mA
Bias: 7 mA
Bias: 8 mA
Bias: 9 mA
Bias: 10 mA
Bias: 11 mA
Bias: 12 mA
Bias: 13 mA
Bias: 14 mA
Bias: 15 mA
Bias: 16 mA
Bias: 17 mA
Bias: 18 mA
Bias: 19 mA
Bias: 20 mA
Bias: 21 mA
Bias: 22 mA
Bias: 23 mA
Bias: 24 mA
Bias: 25 mA
Bias: 26 mA
Bias: 27 mA
Bias: 28 mA
Bias: 29 mA
Bias: 30 mA
Bias: 31 mA
Bias: 32 mA
Bias: 33 mA
Bias: 34 mA
Bias: 35 mA
Bias: 36 mA
Bias: 37 mA
Bias: 38 mA
Bias: 39 mA
Bias: 40 mA
Bias: 41 mA
Bias: 42 mA
Bias: 43 mA
Bias: 44 mA
Bias: 45 mA
Bias: 46 mA
Bias: 47 mA
Bias: 48 mA
Bias: 49 mA
Bias: 50 mA
Bias: 51 mA
Bias: 52 mA
Bias: 53 mA
Bias: 54 mA
Bias: 55 mA
Bias: 56 mA
Bias: 57 mA
Bias: 58 mA
Bias: 59 mA
Bias: 60 mA
Bias: 61 mA
Bias: 62 mA
Bias: 63 mA
Bias: 64 mA
Bias: 65 mA
Bias: 66 mA
Bias: 67 mA
Bias: 68 mA
Bias: 69 mA
Bias: 70 mA
Bias: 71 mA
Bias: 72 mA
Bias: 73 mA
Bias: 74 mA
Bias: 75 mA
Bias: 76 mA
Bias: 77 mA
Bias: 78 mA
Bias: 79 mA
Bias: 80 mA
Bias: 81 mA
Bias: 82 mA
Bias: 83 mA
Bi

In [38]:
meas_df

,Light_engine,Channel,Bias_current_mA,Voltage_V,TEC_C,Ambient_C,LE_C,MPD_mA,Wavelength_nm,Power_dBm,Power_uW,Wavelength_peak_nm,Power_peak_nm,SMSR_dB,SMSR_linewidth_nm,3dB_linewidth_nm,20dB_linewidth_nm
0,241323,4,500.0,0.774076,45.0,34.0000,46.5000,0.46192,1565.00,-51.13,0.007709,1572.19,16.25,60.12,0.43,0.031,0.031
1,241323,4,500.0,0.774076,45.0,34.0000,46.5000,0.46192,1565.01,-49.8,0.010471,1572.19,16.25,60.12,0.43,0.031,0.031
2,241323,4,500.0,0.774076,45.0,34.0000,46.5000,0.46192,1565.02,-50.44,0.009036,1572.19,16.25,60.12,0.43,0.031,0.031
3,241323,4,500.0,0.774076,45.0,34.0000,46.5000,0.46192,1565.03,-50.74,0.008433,1572.19,16.25,60.12,0.43,0.031,0.031
4,241323,4,500.0,0.774076,45.0,34.0000,46.5000,0.46192,1565.04,-50.38,0.009162,1572.19,16.25,60.12,0.43,0.031,0.031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002496,241323,4,0.0,0.775888,45.0,29.8125,44.4375,-0.00046,1584.96,-74.02,0.000040,1572.22,-53.65,0.00,0.00,0.000,0.000
1002497,241323,4,0.0,0.775888,45.0,29.8125,44.4375,-0.00046,1584.97,-62.54,0.000557,1572.22,-53.65,0.00,0.00,0.000,0.000
1002498,241323,4,0.0,0.775888,45.0,29.8125,44.4375,-0.00046,1584.98,-62.78,0.000527,1572.22,-53.65,0.00,0.00,0.000,0.000
1002499,241323,4,0.0,0.775888,45.0,29.8125,44.4375,-0.00046,1584.99,-63.26,0.000472,1572.22,-53.65,0.00,0.00,0.000,0.000


In [39]:
filename = str(Light_engine) + "_" + str(Channel) + "_" + str(T_Start) + ".csv" 
meas_df.to_csv(filename)

### Close Instrument Handles

In [24]:
bias = 0
query_string = "light_engine.set_laser_ma(LEChannel.LE" + str(Channel) + "," + str(bias) + ")"
SSH_ZEUS.write_read(query_string)
tec.set_output_off()

In [27]:
pm.close()
tec.close()
osa.close()
smu.close()

In [28]:
SSH_ZEUS.close()

In [40]:
tec.set_temperature(55)